In [ ]:
from ingestion.pdf_ingestion import extract_text_from_pdf_local
from ingestion.chunking import chunk_text
from embeddings.sbert_embedder import SBERTEmbedder
from vectorstore.faiss_store import FAISSVectorStore
from llm.bedrock_llm import BedrockLLM
from rag.pipeline import RAGPipeline

In [ ]:
# 1. Load & chunk local PDF
text = extract_text_from_pdf_local("sample.pdf")
chunks = chunk_text(text)

In [ ]:
# 2. Embed & index
embedder = SBERTEmbedder()
embeddings = embedder.embed_texts(chunks)

In [ ]:
vs = FAISSVectorStore(dim=len(embeddings[0]))
vs.add_texts(chunks, embeddings)

In [ ]:
# 3. Query
query = "What symptoms are described?"
query_vec = embedder.embed_texts([query])[0]
top_chunks = vs.similarity_search(query_vec)

In [ ]:
# 4. Generate answer
from llm.prompt_templates import qa_prompt_template
llm = BedrockLLM()
prompt = qa_prompt_template(" ".join(top_chunks), query)
print(llm.generate(prompt))
